## 3.1

$$P(am|<s>,I)=\frac{1}{2}=0.5$$
$$P(Sam | I,am)=\frac{1}{2}=0.5$$
$$P(</s>|am Sam)=\frac{1}{1}=1$$
$$P(I|<s>,Sam)=\frac{1}{1}=1$$

## 3.2
$$P(<s> i want chinese food) = P(i|<s>)P(want|i)P(chinese|want)P(food|chinese)\\=0.25\times0.33\times0.0065\times0.52=0.00027885$$

$$P(<s> i want chinese food) = P(i|<s>)P(want|i)P(chinese|want)P(food|chinese)\\=0.25\times0.21\times0.0029\times0.052=0.000007917$$

## 3.3
The first probability is higher without smoothing. Because without applyting smooth, frequent bigrams have higher probability than those in smoothed distribution.

## 3.4


In [34]:
sens = ['<s> I am Sam </s>','<s> Sam I am </s>', '<s> I am Sam </s>', '<s> I do not like green eggs and Sam </s>']
sen_wds = []
for sen in sens:
    sen_wds.append(sen.split())
print(sen_wds)

def add1smooth(sens, pre, cur):
    Vocab = set()
    for sen in sens:
        for wd in sen:
            Vocab.add(wd)
    V = len(Vocab)
    total = target = 0
    for sen in sens:
        for i in range(len(sen)-1):
            if sen[i] == pre:
                total += 1
                if sen[i+1] == cur:
                    target += 1
    return (target+1)/(total+V)
    
print(add1smooth(sen_wds, 'am', 'Sam'))

[['<s>', 'I', 'am', 'Sam', '</s>'], ['<s>', 'Sam', 'I', 'am', '</s>'], ['<s>', 'I', 'am', 'Sam', '</s>'], ['<s>', 'I', 'do', 'not', 'like', 'green', 'eggs', 'and', 'Sam', '</s>']]
0.21428571428571427


## 3.5

In [35]:
def sumadd1smmoth(sens, pres, curs):
    total = 0.0
    for i in range(len(pres)):
        total += add1smooth(sens, pres[i], curs[i])
    return total

sens = ['<s> a b','<s> b b','<s> b a','<s> a a']
print(sumadd1smmoth(sens, ['<s>','<s>','a','b','a','b'],['a','b','a','b','b','a']))

0.8333333333333333


## 3.6

$$P(w_3|w_1,w_2)=\frac{c(w_1,w_2,w_3)}{c(w_1,w_2)}$$
$$P^{*}(w_3|w_1,w_2)=\frac{c(w_1,w_2,w_3)+1}{c(w_1,w_2)+V}$$

## 3.7
$$P\hat(Sam|am)=\lambda_1P(Sam|am)+\lambda_2P(Sam)=0.5\times\frac{2}{3}+0.5\times\frac{4}{25}=0.08+0.33=0.41$$

## 3.8

In [36]:
import collections

def preprocessing(filename):
    f = open(filename,'r',encoding='gb18030', errors='ignore')
    copora = [x.strip().strip(',') for x in f]
    copora = list(filter(None,copora))
    sents = []
    for cop in copora:
        cop = cop.lower().strip('.')
        sent = cop.split(',')
        for sen in sent:
            new_sent = sen.split()
            new_sent = ['<s>'] + new_sent + ['<\s>']
            new_sent = [word.strip('.') for word in new_sent]
            sents.append(new_sent)
    return sents

def bigram(sents, ug_count_dic):
    bg_dic = collections.defaultdict(float)
    for sen in sents:
        for i in range(len(sen)-1):
            if (sen[i], sen[i+1]) not in bg_dic:
                bg_dic[(sen[i],sen[i+1])] = 1
            else:
                bg_dic[(sen[i],sen[i+1])] += 1
    for bi in bg_dic:
        bg_dic[bi] = bg_dic[bi]/ug_count_dic[bi[0]]
    return bg_dic

def unigrams(sents):
    """ 
    @param sents: a list of list of word, each list contain a sentences of a long text file
    @returns (ug_count_dic, unigram_dic): a tuple of two dictionaries with each item as "unigram:count" and "unigram:probability" respectively
    """
    ug_count_dic = collections.defaultdict(int)
    ug_prob_dic = collections.defaultdict(float)
    total = sum(len(sent) for sent in sents)
    for sent in sents:
        for word in sent:
            if word not in ug_count_dic:
                ug_count_dic[word] = 1
            else:
                ug_count_dic[word] += 1
        for word in ug_count_dic:
            ug_prob_dic[word] = ug_count_dic[word]/total
    return ug_count_dic, ug_prob_dic

print(preprocessing('email.txt'))

[['<s>', 'dear', 'members', 'of', 'hkust', '<\\s>'], ['<s>', 'in', 'view', 'of', 'the', 'latest', 'development', 'of', 'the', 'novel', 'coronavirus', 'outbreak', 'in', 'hong', 'kong', '<\\s>'], ['<s>', 'the', "university's", 'management', 'has', 'decided', 'to', 'implement', 'procedures', 'to', 'provide', 'disposable', 'face', 'mask', 'to', 'students', 'and', 'staff', 'who', 'need', 'to', 'be', 'on', 'campus', 'starting', 'from', 'the', 'week', 'of', '10', 'feb', '2020', '<\\s>'], ['<s>', 'campus', 'mask', 'supply', 'scheme', '<\\s>'], ['<s>', '1', 'staff', 'members', 'who', 'have', 'to', 'provide', 'essential', 'services', 'on', 'campus', 'during', 'the', '"work', 'at', 'home', 'period"', 'will', 'be', 'issued', 'a', 'mask', 'on', 'a', 'daily', 'basis', 'departments', 'units', 'are', 'requested', 'to', 'assess', 'the', 'quantities', 'of', 'masks', 'needed', 'and', 'submit', 'requests', 'to', 'respective', "dean's", 'office', '<\\s>'], ['<s>', 'while', 'non-academic', 'offices', 'shoul

## 3.9

In [37]:
sents = preprocessing('email.txt')
ug_count_dic, ug_prob_dic = unigrams(sents)
bi_dic = bigram(sents, ug_count_dic)
# print(ug_prob_dic)
# print(bi_dic)

ug_sorted_dic = {x:v for x,v in sorted(ug_prob_dic.items(), key=lambda item:item[1], reverse=True)}
print(ug_sorted_dic)
bi_sorted_dic = {x:v for x,v in sorted(bi_dic.items(), key=lambda item:item[1], reverse=True)}
print(bi_sorted_dic)

{'<s>': 0.05472636815920398, '<\\s>': 0.05472636815920398, 'the': 0.04975124378109453, 'to': 0.03233830845771144, 'of': 0.02736318407960199, 'and': 0.02736318407960199, 'mask': 0.022388059701492536, 'on': 0.022388059701492536, 'be': 0.01990049751243781, 'a': 0.01990049751243781, 'for': 0.017412935323383085, 'campus': 0.014925373134328358, 'will': 0.012437810945273632, 'is': 0.012437810945273632, 'in': 0.009950248756218905, 'students': 0.009950248756218905, 'staff': 0.009950248756218905, 'supply': 0.009950248756218905, 'at': 0.009950248756218905, 'masks': 0.009950248756218905, 'office': 0.009950248756218905, 'personal': 0.009950248756218905, 'hygiene': 0.009950248756218905, 'who': 0.007462686567164179, 'feb': 0.007462686567164179, '2020': 0.007462686567164179, 'basis': 0.007462686567164179, 'requests': 0.007462686567164179, 'while': 0.007462686567164179, 'offices': 0.007462686567164179, 'collect': 0.007462686567164179, 'their': 0.007462686567164179, 'please': 0.007462686567164179, 'memb

## 3.10 & 3.11

In [38]:
import random

def rand_sent(bi_dic, max_len):
    sent = ['<s>']
    while sent[-1]!='<\s>' and len(sent)+1<max_len:
        pre = sent[-1]
        choices = {bi[1]:prob for bi,prob in bi_dic.items() if bi[0]==pre}
        ran = random.random()
        for item in choices.items():
            ran = ran - item[1]
            if ran < 0:
                sent.append(item[0])
                break
    return sent if sent[-1]=='<\s>' else sent+['<\s>']

def cal_ppl(sent, bi_dic):
    ppl = 1
    for i in range(1,len(sent)-2):
        w1, w2 = sent[i], sent[i+1]
        ppl *= bi_dic[(w1,w2)]
    return pow(ppl, -1/(len(sent)-2))

test_times = 20

total = 0
for i in range(test_times):
    sent = rand_sent(bi_dic, 25)
    ppl = cal_ppl(sent, bi_dic)
    print("sentence %d: ppl = %.2f"%(i+1,ppl))
    print(' '.join([word for word in sent[1:-1]]))
    total += ppl
print("Average ppl: %.2f"%(total/test_times))

sentence 1: ppl = 2.15
safety and may not be on a daily basis departments units are requested to help alleviate the supply of the university's desire to
sentence 2: ppl = 2.34
2020 (wed) while non-academic offices for details and may not be reminded that wearing mask in-person and p/vp offices and present their masks
sentence 3: ppl = 2.61
safety and registration on campus starting from campus during the latest development of the university's desire to stay away from campus mask dissemination
sentence 4: ppl = 2.70
is for information on campus starting from campus community
sentence 5: ppl = 1.92
2020 (wed) while other students can collect their student hall counter
sentence 6: ppl = 2.93
please be sufficient to implement procedures to respective p/vp offices for information on campus starting from the security office on a mask to
sentence 7: ppl = 1.00
health
sentence 8: ppl = 2.24
while the university's management has agreed to respective dean's office
sentence 9: ppl = 2.35
dear mem